<a href="https://colab.research.google.com/github/ukhyun-lee/stock_market_analysis/blob/main/AI%EC%A4%91%EA%B8%89/AI%EC%8B%A4%EC%8A%B5_8%EC%9B%9427%EC%9D%BC(%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from google.colab import files # 데이터 불러오기
file_uploaded=files.upload()   # chap09/data/diabetes.csv 데이터 불러오기

Saving diabetes.csv to diabetes (1).csv


In [7]:
df = pd.read_csv('diabetes.csv')
X = df[df.columns[:-1]]
y = df['Outcome']

X = X.values
y = torch.tensor(y.values)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [8]:
ms = MinMaxScaler()
ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)
y_train =y_train.reshape(-1, 1)
y_test =y_test.reshape(-1, 1)
y_train = ms.fit_transform(y_train)
y_test = ms.fit_transform(y_test)
class customdataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.len = len(self.X)
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    def __len__(self):
        return self.len
train_data = customdataset(torch.FloatTensor(X_train),
                       torch.FloatTensor(y_train))
test_data = customdataset(torch.FloatTensor(X_test),
                       torch.FloatTensor(y_test))

train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        self.layer_1 = nn.Linear(8, 64, bias=True)
        self.layer_2 = nn.Linear(64, 64, bias=True)
        self.layer_out = nn.Linear(64, 1, bias=True)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)

    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        return x

##1.배치 정규화는 ICS를 입력분포가 변화가 되면 학습속도 최적화 영향주므로 막기 위해
##2.역전파시 기울기 소멸 현상 또는 기울기 폭발 현상을 완하시키기 위해
##3.미니 배치 단위로 실행하기 때문에 빠른 학습 가능

In [9]:
epochs = 1000+1
print_epoch = 100
LEARNING_RATE = 1e-2

model = binaryClassification()
model.to(device)
print(model)
BCE = nn.BCEWithLogitsLoss()   #binary_cross_entropy
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
def accuracy(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    return acc

binaryClassification(
  (layer_1): Linear(in_features=8, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [10]:
for epoch in range(epochs):
    iteration_loss = 0.
    iteration_accuracy = 0.

    model.train()
    for i, data in enumerate(train_loader):
        X, y = data
        X, y = X.to(device), y.to(device)
        y_pred = model(X.float()).to(device)
        loss = BCE(y_pred, y.reshape(-1,1).float())

        iteration_loss += loss
        iteration_accuracy += accuracy(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if(epoch % print_epoch == 0):
        print('Train: epoch: {0} - loss: {1:.5f}; acc: {2:.3f}'.format(epoch, iteration_loss/(i+1), iteration_accuracy/(i+1)))

    iteration_loss = 0.
    iteration_accuracy = 0.
    model.eval()
    for i, data in enumerate(test_loader):
        X, y = data
        X, y = X.to(device), y.to(device)
        y_pred = model(X.float()).to(device)
        loss = BCE(y_pred, y.reshape(-1,1).float())
        iteration_loss += loss
        iteration_accuracy += accuracy(y_pred, y)
    if(epoch % print_epoch == 0):
        print('Test: epoch: {0} - loss: {1:.5f}; acc: {2:.3f}'.format(epoch, iteration_loss/(i+1), iteration_accuracy/(i+1)))


Train: epoch: 0 - loss: 0.66371; acc: 59.000
Test: epoch: 0 - loss: 0.66117; acc: 72.500
Train: epoch: 100 - loss: 0.47562; acc: 74.111
Test: epoch: 100 - loss: 0.47003; acc: 77.000
Train: epoch: 200 - loss: 0.38858; acc: 81.444
Test: epoch: 200 - loss: 0.47763; acc: 74.000
Train: epoch: 300 - loss: 0.50443; acc: 77.444
Test: epoch: 300 - loss: 0.53314; acc: 73.250
Train: epoch: 400 - loss: 0.51802; acc: 79.000
Test: epoch: 400 - loss: 0.51043; acc: 71.250
Train: epoch: 500 - loss: 0.41232; acc: 79.222
Test: epoch: 500 - loss: 0.51943; acc: 76.250
Train: epoch: 600 - loss: 0.33659; acc: 86.444
Test: epoch: 600 - loss: 0.50554; acc: 71.250
Train: epoch: 700 - loss: 0.44899; acc: 75.778
Test: epoch: 700 - loss: 0.46836; acc: 77.500
Train: epoch: 800 - loss: 0.48910; acc: 76.333
Test: epoch: 800 - loss: 0.47380; acc: 76.500
Train: epoch: 900 - loss: 0.44462; acc: 77.111
Test: epoch: 900 - loss: 0.45649; acc: 78.750
Train: epoch: 1000 - loss: 0.38202; acc: 82.889
Test: epoch: 1000 - loss: 